# TODO Description of Task

we wan to fine-tune a model for for vote choice prediction.

For the exercises the `TODO` marks places where the the answer should be written/coded.

# TODO Table of Content

# Google Colab Setup
The following is the technical setup required to run the exercises 1, 2, and 3 content, when running the notebook on Google Colab or Kaggle. 

For the fine-tuning we have additional technical setup later.

In [ ]:
# additional colab/kaggle setup

# Importing the sys module to access system-specific parameters and functions
import sys
# Importing the os module to interact with the operating system
import os

# Downloads the data from the GitHub repository of this notebook
def download_data():
    !git clone https://github.com/tobihol/aapor-finetuning.git
    %mv aapor-finetuning/data/ .
    %rm -rf aapor-finetuning/
    return

# Checks if the code is running in a Kaggle environment
def is_running_in_kaggle():
    return 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

# Checks if the code is running in a Google Colab environment
def is_running_in_colab():
    return "google.colab" in sys.modules

# Check if running in Colab or Kaggle and download data accordingly
if is_running_in_colab() or is_running_in_kaggle():
    print("Running on Colab/Kaggle")
    download_data()
else:
    print("Not running in Colab/Kaggle")

# Exercise 1: Model Selection

Choose the right model based on your use case.

Before selecting a model for your task, it’s important to reflect on your goals, resources, and technical environment. Here are key questions to guide your decision:

Do you want to run the model on your own hardware?
- Open-source models (like LLaMA, Mistral, or Qwen) can often be downloaded and run locally. This gives you more control over data privacy and fine-tuning, but also requires sufficient computing power (especially GPU memory) and setup effort.

Do you have funds to use a proprietary model?
- Commercial models like OpenAI’s GPT-4, Anthropic’s Claude, or Google’s Gemini offer strong performance and easy-to-use APIs, but they come with usage costs. This can be worthwhile if you want strong performance without managing infrastructure.

Do you have strong programming skills?
- Open-source models typically require setting up environments (e.g., using PyTorch, HuggingFace Transformers), handling tokenization, managing memory usage, and sometimes fine-tuning. If you’re comfortable with Python and command-line tools, these are manageable. If not, hosted APIs might be easier to work with.

Think about your use case and pick a model. 

**Resources**

The following resources give an overview of available models and how they compare on different metrics of performance and cost.

- The popular benchmark **Chatbot Arena** _(Chiang et al., 2024)_  provides a live updated elo score based on crowd-sourced pairwise comparisons of their responses.
    - Trade-off Plot: https://lmarena.ai/price
- The **Stanford HELM Benchmark** evaluates language models on over 50 tasks, including understanding and reasoning. It provides insights into model performance in terms of accuracy, fairness, and robustness.
    - https://crfm.stanford.edu/helm/
- The **Artificial Analysis** platform provides independent analysis of AI models and API providers. The main metrics are 'intelligence', speed, and price. 
    - https://artificialanalysis.ai/
- The **Open LLM Leaderboard** on Hugging Face runs multiple well established benchmarks on for open-source models hosted on the site. It is most useful to compare smaller open-source LLMs that are not featured on Chatbot Arena or compare different fine-tunes of models. 
    - https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard/

**Notes**

Feel free to make notes here, etc.

**What we are picking**

Our use case is to predict vote choices in the US presidential election based on ANES survey answer. We want to fine-tune a model for _free_ and therefore do not have a lot of computing resources (Google Colab free tier gives access to one T4 GPU). We therefor use the smallest model in the Llama 3 family, **Llama 3.2 1B**, as our model of choice.


# Exercise 2: Excel to JSON

Typical data that is processed by LLMs is assumed to be in JSON format. JSON is a standardized, lightweight, and language-independent format that allows structured data—like text, numbers, lists, or nested objects—to be easily transmitted between systems. Most APIs for LLMs expect JSON because it's both human-readable and machine-readable, making it ideal for sending configuration settings, user input, or model instructions in a clear and organized way.

To get a feeling for how data is structured in JSON as opposed to tabular data (Excel, CSV) we want to convert a small table to JSON in this task.

**Example**

Excel participants table:
|   Age | Gender | Education | Occupation |
|------:|:-------|:----------|:-----------|
|    30 | male   | bachelor  | accountant |
|    24 | female | master    | engineer   |

Converted to JSON:
```json
{
    "participants": [
        {
            "age": 30,
            "gender": "male",
            "education": "bachelor",
            "occupation": "accountant"
        },
        {
            "age": 24,
            "gender": "female",
            "education": "master",
            "occupation": "engineer"
        }
    ]
}
```

**Your Task**

Given the following table of participant answers in a survey:

|   Age | Gender   | State     | Vote Choice   |
|------:|:---------|:----------|:--------------|
|    27 | female   | Louisiana | Trump         |
|    45 | male     | NaN       | Clinton       |
|   NaN | female   | Ohio      | Non-voter     |

What would a corresponding JSON entry look like?

In [22]:
# type up your answer here

json_data = {
    "participants": [
        # TODO
    ]
}

**What is done in practice**

Of course tables are not manually converted in to JSON data in practice Typically python libraries are used to handle data manipulation tasks. In this short course we will use the popular library _pandas_ for parsing tabular data. In the following, two ways of transforming Excel data into JSON data are shown using the library.

Records orientation
- Each row of the table is converted into a JSON object, with the column names as keys and the row values as the corresponding values. This results in a list of JSON objects, where each object represents a row in the table.

Columns orientation
- The table is converted into a JSON object where each key is a column name, and the value is an object containing the index-value pairs for that column. This orientation is useful when you want to access data by columns rather than by rows.


In [ ]:
# Import the popular data manipulation library pandas
import pandas as pd

# Read the Excel file into a DataFrame, using the first column as the index
data = pd.read_excel("data/json_task.xlsx", index_col=0)

# Convert the DataFrame to JSON using records orientation
# Each row is converted into a JSON object with column names as keys
print("--- Records Orientation ---")
print(data.to_json(orient="records"))
print()

# Convert the DataFrame to JSON using columns orientation
# Each column is converted into a JSON object with index-value pairs
print("--- Columns Orientation ---")
print(data.to_json(orient="columns"))
print()

# Exercise 3: Prompt Design

In this exercise, we want to convert our JSON survey participant data, into the structure we will give to the LLM. We will use an _instruction-tuned_ LLM, as they are specifically trained to follow human instructions, as opposed to base models, who predict the next token in a continuous text. Instruction-tuned models use a _chat-format_ as input. These _instruction-tuned_ LLMs follow often follow a similar prompt format, described in the following.

**The OpenAI Chat Format**

The OpenAI JSON chat format has become the standard way to structure conversations with AI models. This format is used not only by OpenAI's models but also by many open-source models available on Hugging Face. This standardized format makes it easy to switch between different models (OpenAI, open-source models on Hugging Face, etc.), maintain conversation history in a structured way, and control the behavior of the model through system messages.

The format consists of a list of messages, each with a "role" and "content":
- **role**: Identifies who is speaking (system, user, or assistant)
- **content**: Contains the actual message text

Overview of the three roles:
- The **system** role provides initial instructions to the model. It sets the behavior and context for the AI assistant. It defines the assistant's role, capabilities, and constraints. The system message not shown to the end user, when using chat bots like ChatGPT, but guides how the model responds.
- The **user** role represents inputs from the human.
- The **assistant** role contains the model's responses.

**Example Conversion**

An example of how to construct a prompt based on survey data in JSON. Here we only consider one participant to make the example more manageable.

Survey data:
```json
{
    "participants": [
        {
            "age": 30,
            "gender": "male",
            "education": "bachelor",
            "occupation": "accountant"
        },
    ]
}
```

An example prompting setup where we want the LLM to be a survey participant that answers questions.
```json
{
    "prompt": [
        {
            "role": "system",
            "content": "You are a survey participant. Reply to the user's question with a short answer."
        },
        {
            "role": "user",
            "content": "What is your age?"
        },
        {
            "role": "assistant",
            "content": "I am 30 years old."
        },
        {
            "role": "user",
            "content": "What is your gender?"
        },
        {
            "role": "assistant",
            "content": "I am a male."
        },
        {
            "role": "user",
            "content": "What is your occupation?"
        },
        {
            "role": "assistant",
            "content": "I am an accountant."
        }
    ]
}
```

**Your Task**

Using the same survey data as provided in the example above. Create a prompting setup that tasks the LLM with predicting the vote choice of a the participant instead.

Survey data:
```json
{
    "participants": [
        {
            "age": 30,
            "gender": "male",
            "education": "bachelor",
            "occupation": "accountant"
        },
    ]
}
```

**Questions to consider:**
- What should the assitant answer look like?
- Should the model act as an expert in a certain field? Who does it substitute?


In [30]:
# type up your answer here

json_prompt = {
    "prompt": [
        # TODO
    ]
}

**Using our prompt with an LLM**

We can test our prompt with a small open-source model. To do this do _not_ include the "assistant" answer JSON object you want the LLM to answer in the prompt. The assistant answer will instead be generated by the LLM, which we choose back in Exercise 1 and we use the instruction-tuned version of the model, as described earlier in this exercise. 

We load the model _Llama-3.2-1B-Instruct_ with the `transformers` library. The transformers library is a popular open-source library developed by Hugging Face that provides pre-trained models for natural language processing (NLP) and other machine learning tasks. It includes tools for fine-tuning, inference optimization, and model deployment. In this notebook, we'll use transformers to load and run the Llama-3.2-1B-Instruct model for text generation.

In [28]:
# Import the pipeline module from the transformers library
# This provides a simple way to use pre-trained models for various NLP tasks
# such as text generation, without having to manually handle model loading and inference
from transformers import pipeline

# id of a model hosted on Hugging Face
model_id = "unsloth/Llama-3.2-1B-Instruct"  # NOTE: we use the unsloth version of the model, because it does not require an API key, as opposed to the official meta-llama version

# Create a pipeline for an instruct model using the json_prompt as input
instruct_pipeline = pipeline(
    "text-generation",  # Specifies the task type for the pipeline (generating text)
    model=model_id,     # Sets which model to use (Llama-3.2-1B-Instruct in this case)
    device_map="auto",  # Automatically determines the best device (CPU/GPU) for running the model
)

After loading the model, we now generate 10 responses to our prompt `json_prompt`. The responses can differ, as they are sampled probabilistically.

In [ ]:
# Generate a response using the instruct model pipeline
responses = instruct_pipeline(json_prompt["prompt"], num_return_sequences=10)

# Print the generated response
print("10 different generated responses:")
[resp["generated_text"][-1] for resp in responses]

# Fine-tuning Pipeline
In the following we go through the steps required to setup a fine-tuning pipeline to impute missing survey data.

In [ ]:
# additional colab/kaggle setup
import sys
import os

def install_dependencies():
    import torch
    if not torch.cuda.is_available():
      print("CUDA is not available. \nPick a GPU before running this notebook. \nGo to 'Runtime' -> 'Change runtime type' to do this. (Colab)")
      return 
    %pip install bitsandbytes
    %pip install accelerate
    %pip install transformers
    %pip install datasets
    %pip install evaluate
    %pip install peft
    %pip install trl
    %pip install evaluate
    %pip install scikit-learn
    %pip install wandb
    return



def is_running_in_kaggle():
    return 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

def is_running_in_colab():
    return "google.colab" in sys.modules

if is_running_in_colab() or is_running_in_kaggle():
    print("Running on Colab/Kaggle")
    install_dependencies()
else:
    print("Not running in Colab/Kaggle")

In [1]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, TaskType
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

seed = 24 # Please set your own favorite seed!
transformers.set_seed(seed)

## Data preperation
We use 2016 American National Election Studies survey data. Specifically, the subset of data Argyle et al. (2022) used in study 2 (https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/JPV20K).

In [ ]:
df_survey = pd.read_csv("data/2016_anes_argyle.csv")
df_survey

In [ ]:
# null values and data types
df_survey.info()

In [4]:
features = [
    "race",
    "discuss_politics",
    "ideology",
    "party",
    "church_goer",
    "age",
    "gender",
    "political_interest",
    "patriotism",
    "state",
]
label = "ground_truth" # this is the vote choice in this dataset

In [ ]:
# we tread missing values as a category 
df_survey_processed = (
    df_survey
    .astype({"age": str})
    .fillna("missing")
)
df_survey_processed

### Train/Test split

Any manipulation of the training data should be done in the step.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_survey_processed, test_size=0.2, random_state=seed)

# we can modify the training data here to do different experiments
# for example excluding republican voters
# leans_republican = df_train["party"].apply(lambda x: "Republican" in x)
# df_train = df_train[~leans_republican]

dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train, preserve_index=False),
    "test": Dataset.from_pandas(df_test, preserve_index=False),
})
dataset

### Prompt Design

We will use an instruction-tuned model and will therefore define an instruction prompt here. Having distinct `user` and `assistant` text. Where the `user` prompt includes all conditioning of the model and the `assistant` text is the expected answer.

In [ ]:
instruction = (
    "Please perform a classification task. "
    "Given the 2016 survey answers from the American National Election Studies, "
    "return which candidate the person voted for. "
    "Return a label from ['Trump', 'Clinton', 'Non-voter'] only without any other text.\n"
)
print(instruction)

In [ ]:
column_name_map = {
    "race": "Race",
    "discuss_politics": "Discusses politics",
    "ideology": "Ideology",
    "party": "Party",
    "church_goer": "Church",
    "age": "Age",
    "gender": "Gender",
    "political_interest": "Political interest",
    "patriotism": "American Flag",
    "state": "State",
    "ground_truth": "Vote",
}


def build_prompt_completion(
    row: dict,
    system_prompt: str = instruction,
) -> list[list[dict]]:
    user_prompt = "\n".join(
        [f"{column_name_map[k]}: {v}" for k, v in row.items() if k != label]
    )
    assistant_prompt = row[label]
    return {
        "prompt": [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
        "completion": [
            {
                "role": "assistant",
                "content": assistant_prompt,
            },
        ],
    }


build_prompt_completion(
    row=dataset["train"][0],
)

In [ ]:
dataset_llm = dataset.map(build_prompt_completion).remove_columns(features+[label])
dataset_llm

## Fine-tuning the model

### Model Setup

We use a very small (1B parameters) model here for demonstration proposes.

In [10]:
# model selection
model_id = "unsloth/Llama-3.2-1B-Instruct"

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    # revision=revision, # NOTE: revision should be set for an reproducible experiment 
    trust_remote_code=True,
)

### Quantization
Quantization reduces the memory required to store the model (Dettmers et al., 2022). Typically a model is stored in 16-bit precision, therefore for a 70B parameter model:

$$\frac{16 \text{ bits}}{8 \text{ bits/byte}} \times 70 \times 10^9 \text{ parameters} = 140 \text{ GB of VRAM}$$

With 4-bit quantization, all parameters are stored in 4-bit precision, reducing the memory requirement to:

$$\frac{4 \text{ bits}}{8 \text{ bits/byte}} \times 70 \times 10^9 \text{ parameters} = 35 \text{ GB of VRAM}$$



In [ ]:
# load model in 4bit
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
    ),
    trust_remote_code=True,
    device_map="auto",
)

model

### LoRA
Low-Rank Adapters (LoRA) are a parameter efficient fine-tuning method (Hu et al., 2021). Instead of finetuning all model weights, LoRA finetunes the weights of the adapter layers only. This requires less memory and allows for faster finetuning.

![LoRA Diagram](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png) 


In [ ]:
lora_rank = 8
lora_alpha = 8

lora_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules="all-linear",
)

lora_config

## Metrics

For simplicity we evaluate on the _first token_ of the LLM generated answer only. If the first token with the highest probability matches the true first token we have classified correctly. We therefore can use typical classification metrics, i.e., _Accuracy_ and _F1 score_.

In [ ]:
import evaluate

hf_metrics = [
    evaluate.load("accuracy"),
    # evaluate.load("f1"),
]

for metric in hf_metrics:
    print(metric.description)

## Additional Training Setup
This is python code is not covered in the workshop and such sets up some additional details for fine-tuning and evaluation.

In [33]:
import wandb

# set pad token id
if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
if getattr(model.config, "pad_token_id") is None:
    model.config.pad_token_id = tokenizer.pad_token_id

labels = df_survey_processed.ground_truth.unique()

first_token_ids = [
    tokenizer.encode(label, add_special_tokens=False)[0]
    for label in labels
]

def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        # Depending on the model and config, logits may contain extra tensors,
        # like past_key_values, but logits always come first
        logits = logits[0]
    # Return highest probability token in answer options
    # return logits[:, :, first_token_ids]
    return logits.argmax(dim=-1)

y_probas = []

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # preds have the same shape as the labels, after the argmax(-1) has been calculated
    # by preprocess_logits_for_metrics but we need to shift the labels
    labels = labels[:, 1:]
    preds = preds[:, :-1]

    # -100 is a default value for ignore_index used by DataCollatorForCompletionOnlyLM
    mask = labels == -100
    labels[mask] = tokenizer.pad_token_id
    preds[mask] = tokenizer.pad_token_id

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    results = {}
    for metric in hf_metrics:
        results |= metric.compute(predictions=preds[~mask], references=labels[~mask])

    return results

## Training the model
To run the training without wandb logging, set `wandb.init(mode='disabled')`.

In [ ]:
from trl import SFTConfig, SFTTrainer
from datetime import datetime

# key hyperparameters
learning_rate = 2e-5
batch_size = 8
epochs = 1 # NOTE you should run multiple epochs in practice

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
dataset_name = "argyle_anes_2016"
run_name = f"{model_id}_{dataset_name}_seed_{seed}_{now}"

# wandb.init(
#     mode='disabled',
# )
wandb.init(
    project="aapor-finetuning",
    name=run_name,
)

training_args = SFTConfig(
    # training
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs, 

    # evaluation
    do_eval=True,
    eval_strategy="steps",
    eval_steps=1 / 3,  # after each third of an epoch

    # logging
    logging_steps=10,
    report_to="wandb",
    run_name=run_name,

    output_dir="./results",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_llm["train"],
    eval_dataset=dataset_llm["test"],
    args=training_args,
    peft_config=lora_config,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

trainer.evaluate()
trainer.train()
trainer.evaluate()

wandb.finish()

All fine-tuning results can be found live at: https://wandb.ai/tobihol/aapor-finetuning